In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [2]:
df = pd.read_csv(r"C:\Training\Academy\Statistics (Python)\Cases\Concrete Strength\Concrete_Data.csv")
df = df.astype(float)
df.head()

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28.0,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28.0,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270.0,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365.0,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360.0,44.30


In [3]:
df.shape

(1030, 9)

In [4]:
X = df.drop('Strength', axis=1)
y = df['Strength']
scaler = MinMaxScaler()
scalerY = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=23)
y_train = scalerY.fit_transform(y_train.values.reshape(-1, 1))
X_train = scaler.fit_transform(X_train)
y_test = scalerY.transform(y_test.values.reshape(-1, 1))
X_test = scaler.transform(X_test)

In [5]:
X_torch = torch.from_numpy(X_train)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([721, 8])
torch.Size([721, 1])


In [6]:
class MLPRegressor(torch.nn.Module):    
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(in_features=num_features, out_features=8)
        self.linear2 = nn.Linear(8,6)
        self.linear3 = nn.Linear(6,4)
        self.linear4 = nn.Linear(4, 3)
        self.linear5 = nn.Linear(3,1)
        self.act = nn.ReLU()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.act(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        x = self.linear5(x)
        output = self.act(x)
        return output

In [7]:
torch.manual_seed(23)
model = MLPRegressor(num_features=X_train.shape[1])

In [8]:
criterion = torch.nn.MSELoss()
# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [9]:
#model[0].weight

In [10]:
y_pred = model(X_torch.float())
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([721, 1])
torch.Size([721, 1])


# Previous Code

### Gradient Descent

for epoch in np.arange(0,5000):
   ### Forward pass: Compute predicted y by passing x to the model
   y_pred = model(X_torch.float())

   ### Compute and print loss
   loss = criterion(y_pred, y_torch.float())
   #print('epoch: ', epoch+1,' loss: ', loss.item())

   ### Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   ### perform a backward pass (backpropagation)
   loss.backward()

   ### Update the parameters
   optimizer.step()
   if epoch % 1000 == 0:
       print('epoch: ', epoch+1,' loss: ', loss.item())

In [11]:
torch.manual_seed(24)
joint_dataset = TensorDataset(X_torch.float(), y_torch.float())
data_loader = DataLoader(dataset=joint_dataset, batch_size=16, shuffle=True)

In [12]:

for epoch in np.arange(0,100):
   for i, batch in enumerate(data_loader, 1):
      # Forward pass: Compute predicted y by passing x to the model
      y_pred_prob = model(batch[0].float())

      # Compute and print loss
      loss = criterion(y_pred_prob, batch[1].view(-1,1).float())
      

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()

      # perform a backward pass (backpropagation)
      loss.backward()

      # Update the parameters
      optimizer.step()
   if epoch%10 == 0:
       print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.07464119046926498
epoch:  11  loss:  0.008765842765569687
epoch:  21  loss:  0.003417316824197769
epoch:  31  loss:  0.005569853819906712
epoch:  41  loss:  0.0018952597165480256
epoch:  51  loss:  0.00028112312429584563
epoch:  61  loss:  0.0016542167868465185
epoch:  71  loss:  0.003929511643946171
epoch:  81  loss:  0.017179356887936592
epoch:  91  loss:  0.010612343437969685


In [13]:
#model = model.eval()
X_torch_test = torch.from_numpy(X_test)
y_pred = model(X_torch_test.float())
y_pred = y_pred.detach().numpy()
y_pred = y_pred.reshape(y_test.shape[0],)
y_pred[:5]

array([0.47651786, 0.10830325, 0.8466482 , 0.18000627, 0.24427693],
      dtype=float32)

In [14]:
y_pred = y_pred.reshape(-1,1)
y_pred_orig = scalerY.inverse_transform(y_pred)
y_pred_orig[:5]

array([[40.58009 ],
       [11.023501],
       [70.29045 ],
       [16.779102],
       [21.938108]], dtype=float32)

In [15]:
y_test_original = scalerY.inverse_transform(y_test)
y_test_original[:5]

array([[36.8 ],
       [11.58],
       [71.7 ],
       [19.52],
       [17.58]])

### Test Set R2 Score

In [16]:
print(r2_score(y_test_original,y_pred_orig))

0.8353014873396419


### Test Set MSE

In [17]:
mse(y_test,y_pred_orig)

1572.9131455825793